In [2]:
pip install catboost 

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
from sklearn.metrics import r2_score
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import MaxAbsScaler, RobustScaler

import tensorflow as tf
tf.random.set_seed(77)  # weight 의 난수값 조정

In [4]:
# 1. 데이터
datasets = fetch_california_housing()  # 다중분류
x = datasets['data']
y = datasets.target

x_train, x_test, y_train,y_test = train_test_split(
    x, y, train_size=0.8, shuffle=True, random_state=42

)
# kfold
n_splits = 5    # 보통 홀수로 들어감
random_state = 42
kfold = KFold(n_splits=n_splits, shuffle=True, 
              random_state=random_state)


# Scaler
scaler = MinMaxScaler()
scaler.fit(x_train)                 # train 은 fit, transform 모두 해줘야 함
x = scaler.transform(x_train) # train 은 fit, transform 모두 해줘야 함
x = scaler.transform(x_test) 

In [7]:
# parameters
param = {
#     'learning_rate': [0.1, 0.01, 0.001],
#     'depth': [3, 5, 6, 9],
#     'l2_leaf_reg': [1, 3, 5, 7],
#     'colsample_bylevel': [1],
#     'n_estimators': [100],
#     'subsample': [3, 5],
#     'border_count': [3]
    'learning_rate': [0.1, 0.5, 1], # controls the learning rate
    'depth': [3, 4, 5], # controls the maximum depth of the tree
    'l2_leaf_reg': [2, 3, 4], # controls the L2 regularization term on weights
    'colsample_bylevel': [0.1, 0.2, 0.3], # specifies the fraction of columns to be randomly sampled for each level
    'n_estimators': [100, 200], # specifies the number of trees to be built
    'subsample': [0.1, 0.2, 0.3], # specifies the fraction of observations to be randomly sampled for each tree
    'border_count': [32, 64, 128],# specifies the number of splits for numerical features
    'bootstrap_type': ['Bernoulli', 'MVS']
}

In [8]:
# 2. 모델
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

cat = CatBoostRegressor() # 성능 가장 좋음
model = GridSearchCV(cat, param, cv=kfold, 
                     refit=True, verbose=1, n_jobs=-1)


# 3. 훈련
import time
start_time = time.time()
model.fit(x_train, y_train)
end_time = time.time() - start_time

print('최적의 파라미터 : ', model.best_params_)
print('최적의 매개변수 : ', model.best_estimator_)
print('best_score : ', model.best_score_)       # 가장 좋은 score
print('model_score : ', model.score(x_test, y_test))    # 실제 데이터를 넣었을 때의 socre
print('걸린 시간 : ', end_time, '초')

Fitting 5 folds for each of 48 candidates, totalling 240 fits
0:	learn: 1.0891268	total: 60.1ms	remaining: 1m
1:	learn: 1.0296697	total: 67.7ms	remaining: 33.8s
2:	learn: 0.9769963	total: 74.7ms	remaining: 24.8s
3:	learn: 0.9298384	total: 81.1ms	remaining: 20.2s
4:	learn: 0.8878288	total: 88.8ms	remaining: 17.7s
5:	learn: 0.8474836	total: 95.1ms	remaining: 15.7s
6:	learn: 0.8100255	total: 101ms	remaining: 14.4s
7:	learn: 0.7818242	total: 108ms	remaining: 13.3s
8:	learn: 0.7540797	total: 114ms	remaining: 12.6s
9:	learn: 0.7326598	total: 121ms	remaining: 12s
10:	learn: 0.7102409	total: 127ms	remaining: 11.5s
11:	learn: 0.6934332	total: 134ms	remaining: 11s
12:	learn: 0.6793271	total: 140ms	remaining: 10.6s
13:	learn: 0.6661036	total: 146ms	remaining: 10.3s
14:	learn: 0.6545354	total: 153ms	remaining: 10s
15:	learn: 0.6400808	total: 159ms	remaining: 9.79s
16:	learn: 0.6258259	total: 166ms	remaining: 9.57s
17:	learn: 0.6154140	total: 172ms	remaining: 9.36s
18:	learn: 0.6081238	total: 177ms

186:	learn: 0.3888347	total: 1.25s	remaining: 5.43s
187:	learn: 0.3882015	total: 1.25s	remaining: 5.42s
188:	learn: 0.3876009	total: 1.26s	remaining: 5.42s
189:	learn: 0.3871085	total: 1.27s	remaining: 5.41s
190:	learn: 0.3866510	total: 1.27s	remaining: 5.39s
191:	learn: 0.3858377	total: 1.28s	remaining: 5.39s
192:	learn: 0.3853512	total: 1.29s	remaining: 5.38s
193:	learn: 0.3850104	total: 1.29s	remaining: 5.37s
194:	learn: 0.3844644	total: 1.3s	remaining: 5.36s
195:	learn: 0.3838610	total: 1.3s	remaining: 5.36s
196:	learn: 0.3835307	total: 1.31s	remaining: 5.35s
197:	learn: 0.3830815	total: 1.32s	remaining: 5.34s
198:	learn: 0.3828072	total: 1.32s	remaining: 5.33s
199:	learn: 0.3820780	total: 1.33s	remaining: 5.32s
200:	learn: 0.3816681	total: 1.33s	remaining: 5.31s
201:	learn: 0.3810977	total: 1.34s	remaining: 5.3s
202:	learn: 0.3805542	total: 1.35s	remaining: 5.29s
203:	learn: 0.3800539	total: 1.35s	remaining: 5.29s
204:	learn: 0.3796793	total: 1.36s	remaining: 5.28s
205:	learn: 0.3

371:	learn: 0.3206050	total: 2.44s	remaining: 4.12s
372:	learn: 0.3203346	total: 2.45s	remaining: 4.12s
373:	learn: 0.3201287	total: 2.46s	remaining: 4.11s
374:	learn: 0.3199494	total: 2.46s	remaining: 4.11s
375:	learn: 0.3197546	total: 2.47s	remaining: 4.1s
376:	learn: 0.3194987	total: 2.48s	remaining: 4.09s
377:	learn: 0.3192164	total: 2.48s	remaining: 4.09s
378:	learn: 0.3190228	total: 2.49s	remaining: 4.08s
379:	learn: 0.3188165	total: 2.5s	remaining: 4.07s
380:	learn: 0.3184424	total: 2.5s	remaining: 4.07s
381:	learn: 0.3180112	total: 2.51s	remaining: 4.06s
382:	learn: 0.3178407	total: 2.52s	remaining: 4.05s
383:	learn: 0.3175488	total: 2.52s	remaining: 4.05s
384:	learn: 0.3173666	total: 2.53s	remaining: 4.04s
385:	learn: 0.3170816	total: 2.54s	remaining: 4.03s
386:	learn: 0.3168657	total: 2.54s	remaining: 4.03s
387:	learn: 0.3167017	total: 2.55s	remaining: 4.02s
388:	learn: 0.3164008	total: 2.56s	remaining: 4.01s
389:	learn: 0.3159933	total: 2.56s	remaining: 4.01s
390:	learn: 0.3

552:	learn: 0.2787894	total: 3.63s	remaining: 2.93s
553:	learn: 0.2786505	total: 3.64s	remaining: 2.93s
554:	learn: 0.2783772	total: 3.64s	remaining: 2.92s
555:	learn: 0.2781277	total: 3.65s	remaining: 2.91s
556:	learn: 0.2780001	total: 3.66s	remaining: 2.91s
557:	learn: 0.2777040	total: 3.66s	remaining: 2.9s
558:	learn: 0.2774812	total: 3.67s	remaining: 2.89s
559:	learn: 0.2773192	total: 3.67s	remaining: 2.89s
560:	learn: 0.2771939	total: 3.68s	remaining: 2.88s
561:	learn: 0.2769739	total: 3.69s	remaining: 2.87s
562:	learn: 0.2767963	total: 3.69s	remaining: 2.87s
563:	learn: 0.2766161	total: 3.7s	remaining: 2.86s
564:	learn: 0.2764062	total: 3.71s	remaining: 2.85s
565:	learn: 0.2762380	total: 3.71s	remaining: 2.85s
566:	learn: 0.2759970	total: 3.72s	remaining: 2.84s
567:	learn: 0.2758228	total: 3.72s	remaining: 2.83s
568:	learn: 0.2756779	total: 3.73s	remaining: 2.83s
569:	learn: 0.2755059	total: 3.74s	remaining: 2.82s
570:	learn: 0.2753686	total: 3.74s	remaining: 2.81s
571:	learn: 0.

735:	learn: 0.2472919	total: 4.82s	remaining: 1.73s
736:	learn: 0.2469996	total: 4.82s	remaining: 1.72s
737:	learn: 0.2468448	total: 4.83s	remaining: 1.71s
738:	learn: 0.2465247	total: 4.83s	remaining: 1.71s
739:	learn: 0.2463698	total: 4.84s	remaining: 1.7s
740:	learn: 0.2462096	total: 4.85s	remaining: 1.69s
741:	learn: 0.2460883	total: 4.85s	remaining: 1.69s
742:	learn: 0.2459489	total: 4.86s	remaining: 1.68s
743:	learn: 0.2457380	total: 4.87s	remaining: 1.67s
744:	learn: 0.2454603	total: 4.87s	remaining: 1.67s
745:	learn: 0.2453548	total: 4.88s	remaining: 1.66s
746:	learn: 0.2452192	total: 4.89s	remaining: 1.65s
747:	learn: 0.2451069	total: 4.89s	remaining: 1.65s
748:	learn: 0.2448863	total: 4.9s	remaining: 1.64s
749:	learn: 0.2447453	total: 4.91s	remaining: 1.64s
750:	learn: 0.2445366	total: 4.91s	remaining: 1.63s
751:	learn: 0.2444342	total: 4.92s	remaining: 1.62s
752:	learn: 0.2443091	total: 4.92s	remaining: 1.61s
753:	learn: 0.2440509	total: 4.93s	remaining: 1.61s
754:	learn: 0.

921:	learn: 0.2214072	total: 6.01s	remaining: 509ms
922:	learn: 0.2212913	total: 6.02s	remaining: 502ms
923:	learn: 0.2212174	total: 6.03s	remaining: 496ms
924:	learn: 0.2211133	total: 6.03s	remaining: 489ms
925:	learn: 0.2210154	total: 6.04s	remaining: 483ms
926:	learn: 0.2209190	total: 6.04s	remaining: 476ms
927:	learn: 0.2206708	total: 6.05s	remaining: 470ms
928:	learn: 0.2204828	total: 6.06s	remaining: 463ms
929:	learn: 0.2203340	total: 6.06s	remaining: 456ms
930:	learn: 0.2201380	total: 6.07s	remaining: 450ms
931:	learn: 0.2200773	total: 6.08s	remaining: 443ms
932:	learn: 0.2198700	total: 6.08s	remaining: 437ms
933:	learn: 0.2197253	total: 6.09s	remaining: 430ms
934:	learn: 0.2196238	total: 6.09s	remaining: 424ms
935:	learn: 0.2195154	total: 6.1s	remaining: 417ms
936:	learn: 0.2193624	total: 6.11s	remaining: 411ms
937:	learn: 0.2192602	total: 6.11s	remaining: 404ms
938:	learn: 0.2191447	total: 6.12s	remaining: 398ms
939:	learn: 0.2190127	total: 6.13s	remaining: 391ms
940:	learn: 0

In [9]:
# 4. 평가, 예측
score = cross_val_score(model, 
                        x_train, y_train, 
                        cv=kfold)   # cv : corss validation
# print('cv acc : ', score)   # kfold 에 있는 n_splits 숫자만큼 나옴 
y_predict = cross_val_predict(model,
                              x_test, y_test,
                              cv=kfold)
r2 = accuracy_score(y_test, y_predict)
print('cv pred r2 : ', r2)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
0:	learn: 1.0878997	total: 4.47ms	remaining: 4.46s
1:	learn: 1.0329666	total: 7.63ms	remaining: 3.81s
2:	learn: 0.9842572	total: 10.6ms	remaining: 3.51s
3:	learn: 0.9420418	total: 13.2ms	remaining: 3.28s
4:	learn: 0.9056837	total: 15.8ms	remaining: 3.14s
5:	learn: 0.8708580	total: 18.4ms	remaining: 3.05s
6:	learn: 0.8408491	total: 21.2ms	remaining: 3.01s
7:	learn: 0.8104669	total: 23.9ms	remaining: 2.96s
8:	learn: 0.7875884	total: 26.6ms	remaining: 2.93s
9:	learn: 0.7665752	total: 29.2ms	remaining: 2.89s
10:	learn: 0.7480541	total: 32.2ms	remaining: 2.89s
11:	learn: 0.7302420	total: 35.1ms	remaining: 2.89s
12:	learn: 0.7097163	total: 37.9ms	remaining: 2.88s
13:	learn: 0.6933756	total: 40.7ms	remaining: 2.86s
14:	learn: 0.6801218	total: 43.5ms	remaining: 2.86s
15:	learn: 0.6698712	total: 46.3ms	remaining: 2.85s
16:	learn: 0.6614578	total: 49.1ms	remaining: 2.84s
17:	learn: 0.6533479	total: 52ms	remaining: 2.83s
18:	learn: 0.6

185:	learn: 0.4320184	total: 579ms	remaining: 2.53s
186:	learn: 0.4313963	total: 582ms	remaining: 2.53s
187:	learn: 0.4310591	total: 585ms	remaining: 2.53s
188:	learn: 0.4307605	total: 587ms	remaining: 2.52s
189:	learn: 0.4303354	total: 590ms	remaining: 2.52s
190:	learn: 0.4298069	total: 593ms	remaining: 2.51s
191:	learn: 0.4295395	total: 596ms	remaining: 2.51s
192:	learn: 0.4290744	total: 599ms	remaining: 2.5s
193:	learn: 0.4287247	total: 601ms	remaining: 2.5s
194:	learn: 0.4279246	total: 604ms	remaining: 2.49s
195:	learn: 0.4273236	total: 608ms	remaining: 2.49s
196:	learn: 0.4270364	total: 610ms	remaining: 2.49s
197:	learn: 0.4265450	total: 614ms	remaining: 2.48s
198:	learn: 0.4261123	total: 616ms	remaining: 2.48s
199:	learn: 0.4257396	total: 619ms	remaining: 2.48s
200:	learn: 0.4254356	total: 622ms	remaining: 2.47s
201:	learn: 0.4249613	total: 625ms	remaining: 2.47s
202:	learn: 0.4243137	total: 628ms	remaining: 2.46s
203:	learn: 0.4237618	total: 631ms	remaining: 2.46s
204:	learn: 0.

395:	learn: 0.3687126	total: 1.16s	remaining: 1.77s
396:	learn: 0.3685190	total: 1.16s	remaining: 1.76s
397:	learn: 0.3683049	total: 1.16s	remaining: 1.76s
398:	learn: 0.3681637	total: 1.17s	remaining: 1.76s
399:	learn: 0.3680319	total: 1.17s	remaining: 1.75s
400:	learn: 0.3678364	total: 1.17s	remaining: 1.75s
401:	learn: 0.3677469	total: 1.18s	remaining: 1.75s
402:	learn: 0.3675718	total: 1.18s	remaining: 1.74s
403:	learn: 0.3672653	total: 1.18s	remaining: 1.74s
404:	learn: 0.3669811	total: 1.18s	remaining: 1.74s
405:	learn: 0.3667534	total: 1.19s	remaining: 1.73s
406:	learn: 0.3665972	total: 1.19s	remaining: 1.73s
407:	learn: 0.3663526	total: 1.19s	remaining: 1.73s
408:	learn: 0.3660529	total: 1.19s	remaining: 1.72s
409:	learn: 0.3659288	total: 1.2s	remaining: 1.72s
410:	learn: 0.3657404	total: 1.2s	remaining: 1.72s
411:	learn: 0.3655607	total: 1.2s	remaining: 1.71s
412:	learn: 0.3653447	total: 1.2s	remaining: 1.71s
413:	learn: 0.3651969	total: 1.21s	remaining: 1.71s
414:	learn: 0.36

610:	learn: 0.3308203	total: 1.75s	remaining: 1.11s
611:	learn: 0.3303937	total: 1.75s	remaining: 1.11s
612:	learn: 0.3302152	total: 1.75s	remaining: 1.1s
613:	learn: 0.3301155	total: 1.75s	remaining: 1.1s
614:	learn: 0.3299996	total: 1.75s	remaining: 1.1s
615:	learn: 0.3299002	total: 1.76s	remaining: 1.1s
616:	learn: 0.3297904	total: 1.76s	remaining: 1.09s
617:	learn: 0.3296319	total: 1.76s	remaining: 1.09s
618:	learn: 0.3295000	total: 1.77s	remaining: 1.09s
619:	learn: 0.3293342	total: 1.77s	remaining: 1.08s
620:	learn: 0.3291577	total: 1.77s	remaining: 1.08s
621:	learn: 0.3289674	total: 1.77s	remaining: 1.08s
622:	learn: 0.3288412	total: 1.78s	remaining: 1.07s
623:	learn: 0.3287450	total: 1.78s	remaining: 1.07s
624:	learn: 0.3285801	total: 1.78s	remaining: 1.07s
625:	learn: 0.3284621	total: 1.79s	remaining: 1.07s
626:	learn: 0.3283190	total: 1.79s	remaining: 1.06s
627:	learn: 0.3281704	total: 1.79s	remaining: 1.06s
628:	learn: 0.3280244	total: 1.79s	remaining: 1.06s
629:	learn: 0.32

824:	learn: 0.3028106	total: 2.33s	remaining: 493ms
825:	learn: 0.3027227	total: 2.33s	remaining: 491ms
826:	learn: 0.3025703	total: 2.33s	remaining: 488ms
827:	learn: 0.3024608	total: 2.33s	remaining: 485ms
828:	learn: 0.3023355	total: 2.34s	remaining: 482ms
829:	learn: 0.3022443	total: 2.34s	remaining: 479ms
830:	learn: 0.3021092	total: 2.34s	remaining: 477ms
831:	learn: 0.3020286	total: 2.35s	remaining: 474ms
832:	learn: 0.3019297	total: 2.35s	remaining: 471ms
833:	learn: 0.3018551	total: 2.35s	remaining: 468ms
834:	learn: 0.3017827	total: 2.35s	remaining: 465ms
835:	learn: 0.3017221	total: 2.36s	remaining: 462ms
836:	learn: 0.3016212	total: 2.36s	remaining: 460ms
837:	learn: 0.3014911	total: 2.36s	remaining: 457ms
838:	learn: 0.3013119	total: 2.37s	remaining: 454ms
839:	learn: 0.3011910	total: 2.37s	remaining: 451ms
840:	learn: 0.3010017	total: 2.37s	remaining: 448ms
841:	learn: 0.3008914	total: 2.37s	remaining: 445ms
842:	learn: 0.3007489	total: 2.38s	remaining: 443ms
843:	learn: 

...

0:	learn: 1.0913565	total: 4.09ms	remaining: 4.09s
1:	learn: 1.0422289	total: 7.35ms	remaining: 3.67s
2:	learn: 0.9997785	total: 8.62ms	remaining: 2.86s
3:	learn: 0.9639586	total: 10.5ms	remaining: 2.63s
4:	learn: 0.9299159	total: 13.3ms	remaining: 2.65s
5:	learn: 0.8998509	total: 17.8ms	remaining: 2.95s
6:	learn: 0.8693401	total: 20.3ms	remaining: 2.87s
7:	learn: 0.8445440	total: 23.6ms	remaining: 2.93s
8:	learn: 0.8183225	total: 25.9ms	remaining: 2.86s
9:	learn: 0.7993355	total: 28.8ms	remaining: 2.85s
10:	learn: 0.7823371	total: 31.3ms	remaining: 2.81s
11:	learn: 0.7657755	total: 33.5ms	remaining: 2.76s
12:	learn: 0.7499271	total: 36ms	remaining: 2.73s
13:	learn: 0.7346745	total: 39ms	remaining: 2.74s
14:	learn: 0.7223456	total: 41.7ms	remaining: 2.74s
15:	learn: 0.7091328	total: 44.2ms	remaining: 2.71s
16:	learn: 0.7006338	total: 46.5ms	remaining: 2.69s
17:	learn: 0.6877286	total: 48.3ms	remaining: 2.64s
18:	learn: 0.6780776	total: 51.2ms	remaining: 2.64s
19:	learn: 0.6713084	total

210:	learn: 0.4096657	total: 380ms	remaining: 1.42s
211:	learn: 0.4089251	total: 382ms	remaining: 1.42s
212:	learn: 0.4081569	total: 383ms	remaining: 1.41s
213:	learn: 0.4075471	total: 384ms	remaining: 1.41s
214:	learn: 0.4071286	total: 385ms	remaining: 1.41s
215:	learn: 0.4065066	total: 387ms	remaining: 1.4s
216:	learn: 0.4058869	total: 388ms	remaining: 1.4s
217:	learn: 0.4053501	total: 389ms	remaining: 1.4s
218:	learn: 0.4047067	total: 391ms	remaining: 1.39s
219:	learn: 0.4043644	total: 392ms	remaining: 1.39s
220:	learn: 0.4038846	total: 393ms	remaining: 1.39s
221:	learn: 0.4033312	total: 394ms	remaining: 1.38s
222:	learn: 0.4025778	total: 395ms	remaining: 1.38s
223:	learn: 0.4020503	total: 397ms	remaining: 1.37s
224:	learn: 0.4015819	total: 398ms	remaining: 1.37s
225:	learn: 0.4010401	total: 399ms	remaining: 1.37s
226:	learn: 0.4003245	total: 400ms	remaining: 1.36s
227:	learn: 0.4000286	total: 402ms	remaining: 1.36s
228:	learn: 0.3997400	total: 403ms	remaining: 1.36s
229:	learn: 0.3

503:	learn: 0.3037665	total: 753ms	remaining: 741ms
504:	learn: 0.3035661	total: 754ms	remaining: 739ms
505:	learn: 0.3032347	total: 755ms	remaining: 737ms
506:	learn: 0.3031042	total: 757ms	remaining: 736ms
507:	learn: 0.3029336	total: 758ms	remaining: 734ms
508:	learn: 0.3026620	total: 759ms	remaining: 732ms
509:	learn: 0.3024368	total: 761ms	remaining: 731ms
510:	learn: 0.3021553	total: 762ms	remaining: 729ms
511:	learn: 0.3019460	total: 763ms	remaining: 727ms
512:	learn: 0.3017165	total: 765ms	remaining: 726ms
513:	learn: 0.3015489	total: 766ms	remaining: 724ms
514:	learn: 0.3012990	total: 767ms	remaining: 723ms
515:	learn: 0.3009318	total: 769ms	remaining: 721ms
516:	learn: 0.3005860	total: 770ms	remaining: 719ms
517:	learn: 0.3004225	total: 771ms	remaining: 718ms
518:	learn: 0.3003331	total: 773ms	remaining: 716ms
519:	learn: 0.3002500	total: 774ms	remaining: 715ms
520:	learn: 0.3000086	total: 775ms	remaining: 713ms
521:	learn: 0.2997156	total: 776ms	remaining: 711ms
522:	learn: 

803:	learn: 0.2438472	total: 1.13s	remaining: 275ms
804:	learn: 0.2437287	total: 1.13s	remaining: 273ms
805:	learn: 0.2435945	total: 1.13s	remaining: 272ms
806:	learn: 0.2435085	total: 1.13s	remaining: 270ms
807:	learn: 0.2434269	total: 1.13s	remaining: 269ms
808:	learn: 0.2433352	total: 1.13s	remaining: 268ms
809:	learn: 0.2431759	total: 1.13s	remaining: 266ms
810:	learn: 0.2430339	total: 1.14s	remaining: 265ms
811:	learn: 0.2428241	total: 1.14s	remaining: 263ms
812:	learn: 0.2426443	total: 1.14s	remaining: 262ms
813:	learn: 0.2425535	total: 1.14s	remaining: 261ms
814:	learn: 0.2423965	total: 1.14s	remaining: 259ms
815:	learn: 0.2422313	total: 1.14s	remaining: 258ms
816:	learn: 0.2420593	total: 1.14s	remaining: 256ms
817:	learn: 0.2419011	total: 1.15s	remaining: 255ms
818:	learn: 0.2417291	total: 1.15s	remaining: 253ms
819:	learn: 0.2415395	total: 1.15s	remaining: 252ms
820:	learn: 0.2414665	total: 1.15s	remaining: 251ms
821:	learn: 0.2412287	total: 1.15s	remaining: 249ms
822:	learn: 

NameError: name 'accuracy_score' is not defined